## Data Simulation

In [1]:
import numpy as np
import matplotlib as plt
from scipy.stats import random_correlation
from numpy.random import default_rng

In [2]:

class SimulatedDataset:
    '''
    description
    '''
    def __init__(self, N,  d, alpha, seed=123):
        self.rng = default_rng(seed)
        self.N = N
        self.d = d
        self.alpha = alpha

        self.X = self.generate_features()

        self.mu0 = self.set_mu0()
        self.mu1 = self.set_mu1()
        self.e = self.set_propensity_fn()
        self.Y0, self.Y1 = self.generate_potential_outcomes()
        self.tau = self.Y1 - self.Y0

        
        self.W = self.generate_treatment()
        self.Y = self.generate_outcome()
        
    # --------------------------
    # CORRELATION STRUCTURE

    def generate_random_eigenvector(self):
        matrix = self.rng.random(size=(self.d, self.d))
        eigvals = np.linalg.eigvals(matrix)
        eigvals = np.abs(eigvals)
        # normalize so eigenvalues sum to d
        eigvals = eigvals / eigvals.sum() * self.d
        return eigvals
    
    def generate_random_correlation_matrix(self):
        eigvals = self.generate_random_eigenvector()
        corr_matrix = random_correlation.rvs(eigvals, random_state=self.rng)
        return corr_matrix
    
    def generate_features(self):
        corr_matrix = self.generate_random_correlation_matrix()
        X = self.rng.multivariate_normal(mean=np.zeros(self.d), cov=corr_matrix, size=self.N)
        return X
    
    # --------------------------
    # STRUCTURAL FUNCTIONS

    def rho(self, x):
        # example heterogeneity function
        return 2 / (1 + np.exp(-5 * (x - 0.35)))

    def set_mu0(self):
        # example control response function (no change) - self.X[:, 0]
        x1 = self.X[:,0]
        x2 = self.X[:,1]
        x3 = self.X[:,2]
        return -0.5 * x3 * self.rho(x1)  * self.rho(x2)
    
    def set_mu1(self):
        # example treated response function (linear) self.X[:, 0] + 0.5 * self.X[:, 1]
        x1 = self.X[:,0]
        x2 = self.X[:,1]
        x3 = self.X[:,2]
        return 0.8 * x3 * self.rho(x1)  * self.rho(x2)
    
    def generate_potential_outcomes(self):
        eps0 = self.rng.normal(0, 1, size=self.N)
        eps1 = self.rng.normal(0, 1, size=self.N)
        Y0 = self.mu0 + eps0
        Y1 = self.mu1 + eps1
        return Y0, Y1
    
    def set_propensity_fn(self):
        '''
        Propensity score function determining probability of treatment = 1
            e(x) = P(W=1|X=x),
        with 'alpha' controlling strength of confounding
        (alpha = 0 is completely random treatment assignment)
        '''
        logits = self.alpha * self.X[:, 0] - 0.6
        return 1 / (1 + np.exp(-logits))
    
    # -------------------------
    # OUTCOME AND TREATMENT

    def generate_treatment(self):
        return self.rng.binomial(1, self.e)
    
    def generate_outcome(self):
        return np.where(self.W == 1, self.Y1, self.Y0)
    
    
    

In [11]:
sample_dataset = SimulatedDataset(1000, 5, 0.5)
sample_dataset.X

array([[-0.22397952,  0.08334883,  0.08279923, -0.06574918, -0.0794817 ],
       [ 1.39578968, -0.55252789,  0.29191761, -0.34562479, -0.89965372],
       [-0.00869309,  0.10532498, -0.17305471, -0.47008639, -1.27314687],
       ...,
       [ 0.51886419,  1.00611509, -0.67144233, -0.75778093, -1.96041459],
       [-0.41228605, -1.27913375,  1.33780791,  0.80433745,  0.24040384],
       [-0.16230506, -1.49438442, -1.29868384, -0.39033459,  0.8643561 ]],
      shape=(1000, 5))

In [12]:

n_treated = sample_dataset.W.sum()
n_control = len(sample_dataset.W) - n_treated
print(n_control, n_treated)

650 350
